In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers


In [2]:
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

11493376/11490434 [==============================] - 0s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [3]:
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense,Conv2D,Input,MaxPooling2D,Flatten
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm

In [4]:
def create_model(activ='relu',dp_rate=0.5,num_layers=1,num_kernals=16,opt='adam'):
  
  model = Sequential()
  model.add(Input(shape=input_shape))
  while num_layers:
    model.add(Conv2D(num_kernals, kernel_size=(3, 3), activation=activ))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    num_layers=num_layers-1
    num_kernals=num_kernals*2  
  model.add(Flatten())
  model.add(Dropout(dp_rate))
  model.add(Dense(num_classes, activation="softmax"))
  model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])    
  return model

In [5]:
test_model=create_model(num_layers=3)
test_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 16)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 1, 1, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 64)                0

In [6]:
import tensorflow as tf
import numpy as np

In [7]:
callback=tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=1e-2,
    patience=5,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
)
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=10, verbose=1)#callbacks=[callback])
para_grid=dict(
activ = ['softmax','relu']
#,dp_rate=np.arange(0,0.5,0.1)
#,num_layers=np.arange(1,3)
#,num_kernals=np.arange(8,32,8)
opt=['SGD', 'RMSprop', 'Adam']
#,epochs=np.arange(10,50,10)
)


In [8]:
grid = GridSearchCV(estimator=model, param_grid=para_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(x_train, y_train)

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Epoch 1/10
6000/6000 [==============================] - 13s 2ms/step - loss: 0.4428 - accuracy: 0.8661
Epoch 2/10
6000/6000 [==============================] - 12s 2ms/step - loss: 0.1457 - accuracy: 0.9552
Epoch 3/10
6000/6000 [==============================] - 11s 2ms/step - loss: 0.1134 - accuracy: 0.9658
Epoch 4/10
6000/6000 [==============================] - 12s 2ms/step - loss: 0.0967 - accuracy: 0.9708
Epoch 5/10
6000/6000 [==============================] - 12s 2ms/step - loss: 0.0846 - accuracy: 0.9739
Epoch 6/10
6000/6000 [==============================] - 12s 2ms/step - loss: 0.0818 - accuracy: 0.9757
Epoch 7/10
6000/6000 [==============================] - 12s 2ms/step - loss: 0.0780 - accuracy: 0.9754
Epoch 8/10
6000/6000 [==============================] - 12s 2ms/step - loss: 0.0769 - accuracy: 0.9764
Epoch 9/10
6000/6000 [==============================] - 11s 2ms/step - loss: 0.0716 - accuracy: 0.9770
Epoch 10/10
6000/6000 [==============================] - 12s 2ms/step - l

In [10]:
print(grid.best_params_)

{'opt': 'Adam'}


In [12]:
grid_result.

{'mean_fit_time': array([112.18494352, 117.78521689, 100.78188396]),
 'mean_score_time': array([4.80533838, 4.73032085, 4.21384271]),
 'mean_test_score': array([0.97438333, 0.96945   , 0.98233332]),
 'param_opt': masked_array(data=['SGD', 'RMSprop', 'Adam'],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'opt': 'SGD'}, {'opt': 'RMSprop'}, {'opt': 'Adam'}],
 'rank_test_score': array([2, 3, 1], dtype=int32),
 'split0_test_score': array([0.97525001, 0.96829998, 0.98329997]),
 'split1_test_score': array([0.97399998, 0.96885002, 0.98215002]),
 'split2_test_score': array([0.97390002, 0.97119999, 0.98154998]),
 'std_fit_time': array([ 5.7746145 ,  2.48967318, 14.32487839]),
 'std_score_time': array([0.37110229, 0.08799434, 0.83288473]),
 'std_test_score': array([0.00061419, 0.00125764, 0.00072609])}

In [14]:
grid.best_estimator_.model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 16)        160       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 16)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2704)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 2704)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                27050     
Total params: 27,210
Trainable params: 27,210
Non-trainable params: 0
_________________________________________________________________


In [15]:
model_trained=grid.best_estimator_.model

In [18]:
model_trained.save('/content/model_trained.h5')